In [0]:
import dlt
from pyspark.sql.functions import col

# Set the Azure Blob Storage access key (optional in DLT if secrets configured)
spark.conf.set(
    "fs.azure.account.key.hexa1.blob.core.windows.net",
    "tEQ86DOREVI9rWneMnt+hKiDIhJ46/Q6F6pnSlUUS3i5jh2bzdiu/eDh7JBuufX0TucDieYs3lyV+ASt6dYDEw=="
)

# 1. Ingest customers.csv
@dlt.table
def customers():
    return (
        spark.read.option("header", True)
            .option("inferSchema", True)
            .option("delimiter", " ")
            .csv("wasbs://images@hexa1.blob.core.windows.net/customers.csv")
    )

# 2. Ingest orders.csv
@dlt.table
def orders():
    return (
        spark.read.option("header", True)
            .option("inferSchema", True)
            .option("delimiter", " ")
            .csv("wasbs://images@hexa1.blob.core.windows.net/orders.csv")
    )

# 3. Join, transform, and enrich data
@dlt.table
def joined_orders():
    orders_df = dlt.read("orders")
    customers_df = dlt.read("customers")
    return (
        orders_df.join(customers_df, on="CustomerID")
        .withColumn("TotalAmount", col("Quantity") * col("Price"))
    )

# 4. Filter high-value orders
@dlt.table
def high_value_orders():
    return dlt.read("joined_orders").filter(col("TotalAmount") > 20000)

# 5. Sorted orders for later export or analysis
@dlt.table
def sorted_orders():
    return dlt.read("joined_orders").orderBy(col("OrderDate").desc())


Name,Type
CustomerID,int
OrderID,int
Product,string
Quantity,int
Price,int
OrderDate,date
Name,string
City,string
Age,int
TotalAmount,int
